# Unsupervised Classification of MNSIT

In this notebook we will be training an autoencoder with MNIST images. This model will be saved and later used for future parts of the project.

---

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras import models

from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Reshape, UpSampling2D, Dropout, Conv2DTranspose, Activation, Concatenate
from keras.models import Model

from keras.datasets import mnist
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import math
import random

import matplotlib.pyplot as plt
import matplotlib.style
import matplotlib as mpl


---

## Step 1: Importing Datasets & Re-shaping Images.

---

In [ ]:
### Downloading the MNIST Dataset ###

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalisation
train_images = train_images / np.max(train_images)
test_images  = test_images / np.max(test_images)

# Dividing images
kTraining_images_1   = train_images
kValidation_images_1 = test_images[:5000]
kTesting_images_1    = test_images[5000:]

kTraining_labels   = train_labels
kValidation_labels = test_labels[:5000]


In [ ]:
### Re-shaping Images ###

def resize_and_convert( images, shape ):
    
    images_expanded = tf.expand_dims( images , -1)
    
    rgb_images = tf.image.grayscale_to_rgb( images_expanded )
    
    resized_images = tf.image.resize( rgb_images , shape)
    
    return resized_images

# ---------------------------------------------------------- #

new_shape = [64, 64]

kTraining_images   = resize_and_convert( kTraining_images_1, new_shape )
kValidation_images = resize_and_convert( kValidation_images_1, new_shape )
kTesting_images    = resize_and_convert( kTesting_images_1, new_shape )


---

## Step 2: Creating and Training the Autoencoder.

- No optimisation techniques applied at first.

---

In [ ]:
### Autoencoder Model ###

# Needed for decoder #
shape_before_flattening = (4, 4, 256)

# -------------------------------------------------------------------------------- #

input_img_y = Input(shape=(64, 64, 3))

# Encoder #
y = Conv2D( 32, (3, 3), activation='relu', padding='same', strides=2)(input_img_y)
y = Conv2D( 64, (3, 3), activation='relu', padding='same', strides=2)(y)
y = Conv2D( 128, (3, 3), activation='relu', padding='same', strides=2)(y)
y = Conv2D( 256, (3, 3), activation='relu', padding='same', strides=2)(y)

# Latent layers #
y = Flatten()(y)
y = Dense( 32, activation='relu')(y)
y = Dense( np.prod(shape_before_flattening) )(y)
y = Reshape( shape_before_flattening )(y)

# Decoder #
y = Conv2DTranspose( 256, (3, 3), activation='relu', padding='same', strides=2)(y)
y = Conv2DTranspose( 128, (3, 3), activation='relu', padding='same', strides=2)(y)
y = Conv2DTranspose( 64, (3, 3), activation='relu', padding='same', strides=2)(y)
y = Conv2DTranspose( 32, (3, 3), activation='relu', padding='same', strides=2)(y)

output_y = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(y)

# ----------- Autoencoder model ----------- #
autoencoder_y = Model( input_img_y, output_y )

autoencoder_y.compile(optimizer="adam",
                      loss="mean_squared_error",
                      metrics=['mean_squared_error','accuracy'])

autoencoder_y.summary()


In [ ]:
### Training model ###

history = autoencoder_y.fit( kTraining_images, kTraining_images,
                            epochs=5,
                            validation_data=( kValidation_images, kValidation_images ))


In [ ]:
### PLOTTING ###

# mean squared error data
loss = history.history["mean_squared_error"]
val_loss = history.history["val_mean_squared_error"]

# x axis
epochs = range(1, len(loss) + 1)

# ---------------------------- Plotting ---------------------------- #

fig1 = plt.figure(figsize=(15,7.5))
ax2 = fig1.add_subplot(1,1,1)

ax2.plot(epochs, loss, "ro", label="Training")
ax2.plot(epochs, val_loss, "r", label="Validation")
ax2.legend()
ax2.grid()

ax2.set_ylabel('Mean squared error')
ax2.set_xlabel('Epochs')

y_labels = np.arange(0,0.016,0.001)  # Define your labels here
ax2.set_yticks(y_labels);


---

## Step 3: Testing and Saving model.

---

In [ ]:
### Testing Quality of Autoencoder Reconstruction ###

# choosing test images
constructed_image = kTesting_images[ 3 ]
constructed_image = np.expand_dims( constructed_image, axis=0)

# applying autoencoder model
reconstructed_image = autoencoder_y.predict(constructed_image)

# ---------------------------- Plotting ---------------------------- #

fig1 = plt.figure()

ax1 = fig1.add_subplot(1,2,1)
ax2 = fig1.add_subplot(1,2,2)

ax1.imshow( constructed_image[0] , cmap='gray')
ax2.imshow( reconstructed_image[0] , cmap='gray')

ax1.set_title('Original')
ax2.set_title('Re-constructed');

In [ ]:
### ----------------------- SAVING MODEL ----------------------- ###

# autoencoder_y.save('autoencoder_model_mnist')


---